<a href="https://colab.research.google.com/github/esemoney/data-ese/blob/main/Copy_of_Query_Blockchain_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Query Blockchain Data
**Purpose:** Learn how to easily query raw blockchain data via APIs from leading blockchain analytics platforms.
1. [Flipside](https://flipsidecrypto.xyz/)
2. [Footprint](https://www.footprint.network/)
3. [Dune](https://dune.com/home)
4. [Luabase](https://luabase.com/)
5. [Google BigQuery](https://console.cloud.google.com/marketplace/browse)

Note: All examples pull raw data for 10 random transactions from Ethereum.

In [ ]:
import requests
import json
import pandas as pd
import os

## 1. [Flipside](https://flipsidecrypto.xyz/)
### Method #1 - API
**Steps:**
1. Create a free Flipside account.
2. Create a query.
3. Copy & paste the API link for the query (see "API" button in top right part of editor).

In [ ]:
# Example Query: https://app.flipsidecrypto.com/velocity/queries/350df4c5-db0e-4c8e-ac0d-1b845ecbe7a0
query_api_url = "https://node-api.flipsidecrypto.com/api/v2/queries/350df4c5-db0e-4c8e-ac0d-1b845ecbe7a0/data/latest"
results_json = requests.get(query_api_url).json()
results_df = pd.DataFrame.from_dict(results_json)
results_df.head()

### Method #2 - SDK

**Steps**:
1. Create a free Flipside account.
2. Generate a SDK NFT to get API key [here](https://sdk.flipsidecrypto.xyz/shroomdk).
3. `pip install shroomdk`


**Supporting Links:**
*   https://docs.flipsidecrypto.com/shroomdk-sdk/python
*   https://docs.flipsidecrypto.com/shroomdk-sdk/examples

In [ ]:
pip install shroomdk

In [ ]:
from shroomdk import ShroomDK

def query_flipside_sdk(query_string, sdk_key):
    # Created by https://twitter.com/0xdatawolf
    sdk = ShroomDK(sdk_key)
    
    # Query results page by page and saves the results in a list
    # If nothing is returned then just stop the loop and start adding the data to the dataframe
    result_list = []
    for i in range(1,11): # max is a million rows @ 100k per page
        data=sdk.query(query_string,page_size=100000,page_number=i)
        if data.run_stats.record_count == 0:  
            break
        else:
            result_list.append(data.records)
        
    # Loops through the returned results and adds into a pandas dataframe
    result_df=pd.DataFrame()
    for idx, each_list in enumerate(result_list):
        if idx == 0:
            result_df=pd.json_normalize(each_list)
        else:
            result_df=pd.concat([result_df, pd.json_normalize(each_list)])

    return result_df

# Creds/keys
flipside_sdk_key = ""

# Query
query_string = """
  select *
  from ethereum.core.fact_transactions
  limit 10
"""

results_df = query_flipside_sdk(query_string, flipside_sdk_key)
results_df.head()

## 2. [Footprint](https://www.footprint.network/)

**Steps:**
1. Create a free Footprint account.
2. Generate API Key [here](https://www.footprint.network/account/developer).

**Supporting Links:**
* https://docs.footprint.network/reference/introduction

In [ ]:
# Creds/keys
footprint_api_key = ""

# Query
query_string = """
  select * 
  from ethereum_transactions
  limit 10
"""

url = "https://api.footprint.network/api/v1/native"

payload = {"query": query_string}
headers = {
    "accept": "application/json",
    "API-KEY": footprint_api_key,
    "content-type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)
results_json = json.loads(response.text)["data"]
results_df = pd.DataFrame.from_dict(results_json)
results_df.head()

## 3. [Dune](https://dune.com/home)

### Method #1 - Using a third party: https://github.com/itzmestar/duneanalytics

**Steps:**
1. Create a free Dune account.
2. Create a query.
3. `pip install duneanalytics`

**Supporting Links:**
* Official Dune API in private beta. Coming soon... 
 * https://dune.com/docs/api/quick-start/api-py/
* Other Third party python apis:
 * https://pypi.org/project/dune-analytics/
 * https://pypi.org/project/duneapi/
 * https://pypi.org/project/dunebuggy/

In [ ]:
pip install duneanalytics

In [ ]:
from duneanalytics import DuneAnalytics

# Example Query: https://dune.com/queries/1438636 

# Creds/keys
dune_username = ''
dune_password = ''

dune = DuneAnalytics(dune_username, dune_password)
dune.login()
dune.fetch_auth_token()
result_id = dune.query_result_id(query_id=1438636)
data = dune.query_result(result_id)

result_df=pd.DataFrame()
for i in data['data']['get_result_by_result_id']:
  result_df = pd.concat([result_df, pd.json_normalize(i['data'])])

result_df.head()

## 4. [Luabase](https://luabase.com/)

**Steps**:
1. Create a free Luabase account.
2. Create a new notebook & create a query.
3. Click "View API", which auto generates Python code.
4. Generate API key.

**Supporting Links:**
* https://luabase.notion.site/API-b86c02f5fc244041bcd365c02628fa72#f570519b70fd4457b330a34c0d5ba45c

In [ ]:
# Option A: Pre Made Query

# Example Notebook w/ Query: https://luabase.com/notebook/184d77c4f73b41839224ae5023046cc6

# Creds/keys
luabase_api_key = ""

url = "https://q.luabase.com/run"
payload = {
    "block": {"data_uuid": "184d77c4f73b41839224ae5023046cc6"},
    "api_key": luabase_api_key
}
headers = {"content-type": "application/json"}
response = requests.request("POST", url, json=payload, headers=headers)
data = response.json()["data"]
results_df = pd.json_normalize(data)
results_df.head()

In [ ]:
# Option B: Dynamic Query

luabase_api_key = ""

url = "https://q.luabase.com/run"

sql = '''
select * 
from ethereum.transactions
limit 10
'''

payload = {
  "block": {
    "details": {
      "sql": sql,
    }
  },
	  "api_key": luabase_api_key,
}
headers = {"content-type": "application/json"}
response = requests.request("POST", url, json=payload, headers=headers)
data = response.json()["data"]
results_df = pd.json_normalize(data)
results_df.head()

## 5. [Google BigQuery](https://console.cloud.google.com/marketplace/browse)

**Steps:**
1. Create a Google Cloud Platform (GCP) account.
2. Use the [Cloud Resource Manager](https://console.cloud.google.com/cloud-resource-manager) to create a Cloud Platform project.
3. [Enable billing](https://support.google.com/cloud/answer/6293499#enable-billing) for the project.
4. [Enable BigQuery](https://console.cloud.google.com/flows/enableapi?apiid=bigquery) APIs for the project.

**Supporting Links:**
* https://cloud.google.com/blog/products/data-analytics/introducing-six-new-cryptocurrencies-in-bigquery-public-datasets-and-how-to-analyze-them

In [ ]:
from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()

# Creds/keys
google_project_id = ""

# Query
query_string = """
  SELECT * 
  FROM `bigquery-public-data.crypto_ethereum.transactions` 
  WHERE DATE(block_timestamp) = "2022-10-21" 
  LIMIT 10
"""

client = bigquery.Client(project=google_project_id)
results_df = client.query(query_string).to_dataframe()
results_df.head()